In [ ]:
## Analysis of the reconstrcutions after eliminating some channels when training.
## This is the analysis of the results coming from the flies: 
### root = C:\Users\David\Documents\GitHub\encoding\scripts\wm_representation\functions\IEM\Controls\trial_by_trial\
########## root + trainT_testT_wm3_remove_permissive.py
########## root + trainT_testT_wm3_remove_restrictive.py

In [1]:
from linares_plot import *
import sys, os
import statsmodels.formula.api as smf
path_tools = 'C:\\Users\\David\\Documents\\GitHub\\encoding\\scripts\\wm_representation\\functions\\IEM\\tools'
sys.path.insert(1, path_tools)
from timecourse import *
from timecourse_c import *
from scipy import stats
from mlxtend.evaluate import permutation_test

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
## Permissive --> 28 of 36 channels

In [7]:
##Load files

#### Reconstructions
recons_path =  'C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\recs_IEM_trainT_testT_wm3_permissive.npy' 
Reconstructions = np.load(recons_path, encoding='latin1')
print('Reconstructions')
print(np.shape(Reconstructions))
print(np.shape(Reconstructions[10]))
print('')

#### behavior
beh_path =  'C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\trainT_testT_behaviour_permissive.xlsx' 
xl = pd.ExcelFile(beh_path)
sheet_names = xl.sheet_names
print('Behavior')
print(len(sheet_names))
beh_x = pd.read_excel(beh_path, sheet_name=sheet_names[10])
print(np.shape(beh_x))


Reconstructions
(24,)
(372, 16, 28)

Behavior
24
(372, 46)


In [8]:
## Indexes equivalences

Subjects=['d001', 'n001', 'b001', 'r001', 's001', 'l001']
brain_regions = ['visual','ips', 'pfc', 'broca']

index = 0
for Subject in Subjects:
    for Brain_region in brain_regions:
        print(index, Subject, Brain_region)
        index += 1
        

0 d001 visual
1 d001 ips
2 d001 pfc
3 d001 broca
4 n001 visual
5 n001 ips
6 n001 pfc
7 n001 broca
8 b001 visual
9 b001 ips
10 b001 pfc
11 b001 broca
12 r001 visual
13 r001 ips
14 r001 pfc
15 r001 broca
16 s001 visual
17 s001 ips
18 s001 pfc
19 s001 broca
20 l001 visual
21 l001 ips
22 l001 pfc
23 l001 broca


In [9]:
## Heatmap function
sep_channels=10
adjusted_size_contant = 48.519

pos_channels = np.arange(sep_channels/2,360,sep_channels)
pos_channels = [round(pos_channels[i],3) for i in range(0, len(pos_channels))]

pos_channels2 = np.arange(0,360,0.5)
pos_channels2 = [round(pos_channels2[i],3) for i in range(0, len(pos_channels2))]

def posch1_to_posch2(ch_1):
    return np.where(np.array(pos_channels2) == pos_channels[ch_1])[0][0]


def Representation_heatmap(df, condition='1_0.2', min_=None, max_=None, ref_angle=180, TR=2.335, bin_=0.5, fs1=20, fs2=15):
    ##
    ##
    ##### pal_cyan = sns.color_palette("RdBu_r", n_colors=200)[40:] #RdBu_r
    pal_cyan = 'viridis'
    #
    if min_==None:
        min_ = np.round(df.mean().mean()-4*df.std().mean(),2)
    if max_==None:
        max_ = np.round(df.mean().mean()+4*df.std().mean(),2)
    #
    middle = np.round( min_ + ((abs(max_)+abs(min_))/2), 2)
    ##
    ##
    
    plt.figure(figsize=(8,6))
    ax = sns.heatmap(df, cmap=pal_cyan, 
                     vmin=min_, 
                     vmax=max_,
                     cbar=True, 
                cbar_kws={"shrink": .82, 'ticks' : [min_, middle, max_], 'label': 'reconstruction (a.u)'})
    ax.figure.axes[-1].yaxis.label.set_size(fs1)
    ax.figure.axes[-1].tick_params(labelsize=fs2)
    
    plt.gca().set_ylabel('')
    plt.yticks([posch1_to_posch2(4), posch1_to_posch2(13), posch1_to_posch2(22), posch1_to_posch2(31)],['45','135','225', '315'])

    dims=np.shape(df)
    dimN = dims[0]
    plt.gca().set_ylim(dimN, -45)
    
    plt.plot([0.5, 0.5], [posch1_to_posch2(18),posch1_to_posch2(18)], markersize=20, color='grey', marker='>')
    plt.plot([1, 23.5], [posch1_to_posch2(18),posch1_to_posch2(18)], color='grey', linestyle='--', linewidth=1.5)
    
    ################################
    presentation_period= 0.35 #stim presnetation time
    presentation_period_cue=  0.50 #presentation of attentional cue time
    pre_stim_period= 0.5 #time between cue and stim
    resp_time = 4  #time the response is active
    start_hrf = 4  #start of the Hemodynamic response (4seconds)
    if condition == '1_0.2':
        condition_title = 'o:1, d:0.2'
        delay1 = 0.2
        delay2 = 11.8
        cue=0
        t_p = cue + presentation_period_cue + pre_stim_period 
        d_p = t_p + presentation_period +delay1 
        r_t = d_p + presentation_period + delay2
        xlim = [1/TR, 25/TR]
    elif condition=='1_7':
        condition_title = 'o:1, d:7'
        delay1 = 7
        delay2 = 5
        cue=0
        t_p = cue + presentation_period_cue + pre_stim_period 
        d_p = t_p + presentation_period +delay1 
        r_t = d_p + presentation_period + delay2
        xlim = [1/TR, 25/TR]
    elif condition=='2_0.2':
        condition_title = 'o:2, d:0.2'
        delay1 = 0.2
        delay2 = 12
        cue=0
        d_p = cue + presentation_period_cue + pre_stim_period 
        t_p = d_p + presentation_period +delay1 
        r_t = t_p + presentation_period + delay2   
        xlim = [1/TR, 25/TR]
    elif condition=='2_7':
        condition_title = 'o:2, d:7'
        delay1 = 7
        delay2 = 12
        cue=0
        d_p = cue + presentation_period_cue + pre_stim_period 
        t_p = d_p + presentation_period +delay1 
        r_t = t_p + presentation_period + delay2
        xlim = [1/TR, 30/TR]
    ##
    ##
    t_p1 = (start_hrf + t_p) ## strat of target (time)
    d_p1 = (start_hrf + d_p) ##strat of didtractor (time)
    r_p1=  (start_hrf + r_t) ## start of response (time)
    ##
    plt.plot([t_p1/TR +bin_,t_p1/TR+bin_], [-15,-15], markersize=10, color='grey', marker='v')
    plt.plot([d_p1/TR +bin_,d_p1/TR+bin_], [-15,-15], markersize=10, color='darkred', marker='v')
    plt.plot([r_p1/TR +bin_,r_p1/TR+bin_], [-15,-15], markersize=10, color='y', marker='v')
    plt.gca().set_xlim(xlim)
    
    plt.xticks(fontsize=fs2)
    plt.yticks(fontsize=fs2)
    plt.gca().set_ylabel('angle ($^\circ$)', fontsize=fs1)
    plt.gca().set_xlabel('time (s)', fontsize=fs1)
    #plt.title(condition_title)
    
    plt.show()


In [10]:
#### Plot heatmap of one condition in one subject

idx_ = 4 ##  0 --> n001 visual

reconstr_trials = np.array(Reconstructions[idx_])
beh =  pd.read_excel(beh_path, sheet_name=sheet_names[idx_])

print(np.shape(reconstr_trials))
print(np.shape(beh))

(372, 16, 28)
(372, 46)


In [11]:
Condition_ = '1_0.2'
bool_condition = np.array(beh.Condition==Condition_)
print(len(bool_condition))
print(sum(bool_condition))

reconstr_trials_cond = reconstr_trials[bool_condition]
print(np.shape(reconstr_trials_cond))

#mean_reconstr_cond = reconstr_trials_cond.mean(axis=0)
#print(np.shape(mean_reconstr_cond))



372
96
(96, 16, 28)


In [17]:
beh_cond = beh.loc[beh['Condition']==Condition_]
print(np.shape(beh_cond))
targets = beh_cond.T_alone.values
print(len(targets))

(96, 46)
96


In [36]:
Channels_ = ['ch_2', 'ch_3', 'ch_4', 'ch_5', 'ch_6', 'ch_7', 'ch_8', 'ch_11','ch_12', 'ch_13', 'ch_14', 'ch_15', 'ch_16', 'ch_17', 
    'ch_20', 'ch_21', 'ch_22', 'ch_23', 'ch_24', 'ch_25', 'ch_26', 'ch_29','ch_30', 'ch_31', 'ch_32', 'ch_33', 'ch_34', 'ch_35']
channels = np.array([int(Channels_[n].split('_')[1]) for n in range(len(Channels_))])
channels

array([ 2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22,
       23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 35])

In [32]:
centers_channels = np.arange(5,365,10)
number_channels = np.arange(1,37,1)
equivalence_ = pd.DataFrame({'angle centered': centers_channels, 'number ch':number_channels})
equivalence_

,angle centered,number ch
0,5,1
1,15,2
2,25,3
3,35,4
4,45,5
5,55,6
6,65,7
7,75,8
8,85,9
9,95,10


In [33]:
np.shape(reconstr_trials_cond[0])

(16, 28)

In [ ]:
def Add_channels_28(reconstruction, channels):
    ##add channels with activity = 0 for reconstructions of 